<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [10]:
# Install spark
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [11]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import findspark
findspark.init()

In [12]:
# Start session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("E-commerce Search Terms Analysis") \
    .getOrCreate()


In [14]:
# Download The search term dataset from the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2024-09-13 14:49:11--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.004s  

2024-09-13 14:49:12 (52.5 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [20]:
# Load the csv into a spark dataframe
search_terms_df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)


In [21]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)
num_rows = search_terms_df.count()
num_cols = len(search_terms_df.columns)

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")


Number of rows: 10000
Number of columns: 4


In [22]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)
top_5_rows = search_terms_df.show(5)


+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [23]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)
from pyspark.sql.functions import col, count

top_terms_df = search_terms_df.groupBy("searchterm").agg(count("*").alias("count")).orderBy(col("count").desc())
top_5_terms = top_terms_df.show(5)


[Stage 10:===================================================>  (189 + 8) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [24]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg
gaming_laptop_count = search_terms_df.filter(search_terms_df.searchterm == "gaming laptop").count()
print(f"Number of times 'gaming laptop' was searched: {gaming_laptop_count}")


Number of times 'gaming laptop' was searched: 499


In [25]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)
from pyspark.sql.functions import col, count

top_terms_df = search_terms_df.groupBy("searchterm").agg(count("*").alias("count")).orderBy(col("count").desc())
top_5_terms = top_terms_df.show(5)



[Stage 14:=========================================>           (157 + 11) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [30]:
# The pretrained sales forecasting model is available at  the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar -xzvf model.tar.gz

--2024-09-13 14:55:28--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz.1’

model.tar.gz.1      100%[===================>]   1.46K  --.-KB/s    in 0s      

2024-09-13 14:55:28 (13.4 MB/s) - ‘model.tar.gz.1’ saved [1490/1490]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_predic

In [33]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)
from pyspark.ml.regression import LinearRegressionModel

# Path to the extracted model directory
model_directory = "sales_prediction.model"

# Load the model as a LinearRegressionModel
model = LinearRegressionModel.load(model_directory)

# Print the model details to verify
print(model)




LinearRegression_6d5736f3dbe7


In [35]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import Row
from pyspark.ml.regression import LinearRegressionModel

model_directory = "sales_prediction.model"
model = LinearRegressionModel.load(model_directory)

# Create a DataFrame for prediction
data_for_prediction = spark.createDataFrame([Row(year=2023)])

# Assemble the features into a single column
assembler = VectorAssembler(inputCols=["year"], outputCol="features")
data_with_features = assembler.transform(data_for_prediction)

# Predict sales
predictions = model.transform(data_with_features)

# Show the predictions
predictions.show()


+----+--------+------------------+
|year|features|        prediction|
+----+--------+------------------+
|2023|[2023.0]|175.16564294006457|
+----+--------+------------------+



24/09/13 14:57:25 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/09/13 14:57:25 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
